In [23]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
# from sympy.abc import M, I
import math

from sympy import print_latex
from IPython.display import display

init_vprinting()

## Rotation matrices

In [2]:
M, m = sp.symbols('M m', real=True, positive=True)
L = sp.symbols('L', real=True, positive=True)

phi, theta =  dynamicsymbols('phi, theta')
beta = sp.symbols('beta', real=True, positive=True)

g = sp.Symbol('g', real=True)
t = sp.Symbol('t')

# Rotation matrices 
B_R_P = sp.rot_axis3(phi)
A_R_B = sp.rot_axis1(beta)
I_R_A = sp.rot_axis3(theta)

## Angular velocities

### Plate

In [3]:
B_omega  = sp.Matrix([[0],[0],[phi.diff(t)]])
A_omega  = sp.Matrix([[0],[0],[theta.diff(t)]])

A_omega_plate = A_omega + sp.rot_axis1(sp.pi/2 - beta)  * B_omega
A_omega_plate

⎡     0      ⎤
⎢            ⎥
⎢  cos(β)⋅φ̇  ⎥
⎢            ⎥
⎣sin(β)⋅φ̇ + θ̇⎦

### Rods

In [4]:
A_omega_rod1 = sp.Matrix([[0],[0],[theta.diff(t)]])

Bdash_omega_rod2  = sp.Matrix([[0],[0],[phi.diff(t)]])
A_omega_rod2 = A_omega_rod1 + sp.rot_axis1(sp.pi/2 - beta)  * Bdash_omega_rod2

## Linear Velocities

### Plate

In [5]:
A_v_oodash = sp.Matrix([[-L*theta.diff(t)],[0],[0]])
B_r_podash = sp.rot_axis1(-(sp.pi/2-beta)).T *  sp.rot_axis3(-phi).T * sp.Matrix([[0],[L/2],[L/2]])

# # for verification 
# B_r_podash.subs({beta:sp.pi/2, phi:sp.pi/2})

A_v_po =  A_v_oodash  + A_omega_plate.cross(B_r_podash)
A_v_po.applyfunc(sp.simplify)

⎡-L⋅(sin(β)⋅cos(φ)⋅θ̇ + cos(β)⋅θ̇ + cos(φ)⋅φ̇ + 2⋅θ̇) ⎤
⎢─────────────────────────────────────────────────⎥
⎢                        2                        ⎥
⎢                                                 ⎥
⎢             L⋅(sin(β)⋅φ̇ + θ̇)⋅sin(φ)             ⎥
⎢             ───────────────────────             ⎥
⎢                        2                        ⎥
⎢                                                 ⎥
⎢               -L⋅sin(φ)⋅cos(β)⋅φ̇                ⎥
⎢               ───────────────────               ⎥
⎣                        2                        ⎦

In [6]:
# Sanity checks

# check whether x and z velocoty components due to phi rotation form circular path  and omega rotation is added
A_v_po.subs({beta:0}).applyfunc(sp.simplify)

⎡-L⋅(cos(φ)⋅φ̇ + 3⋅θ̇) ⎤
⎢────────────────────⎥
⎢         2          ⎥
⎢                    ⎥
⎢     L⋅sin(φ)⋅θ̇     ⎥
⎢     ──────────     ⎥
⎢         2          ⎥
⎢                    ⎥
⎢    -L⋅sin(φ)⋅φ̇     ⎥
⎢    ────────────    ⎥
⎣         2          ⎦

### Rods

In [7]:
A_v_rod1 =  sp.Matrix([[-L*theta.diff(t)/2],[0],[0]])
A_v_rod2 = sp.Matrix([[-theta.diff(t) * (L + (L/2* sp.cos(beta))) ],[0],[0]])

# Lagrangian

In [8]:
def norm_sqrd(v):
    return sp.simplify(sp.Pow(v[0],2) + sp.Pow(v[1],2) + sp.Pow(v[2],2))

def elementwise_sqrd(v):
    return sp.Matrix([[sp.Pow(v[0],2)],[sp.Pow(v[1],2)],[sp.Pow(v[2],2)]]) 

## Kinetic energy

### Kinetic energy of plate

In [9]:
I_xx = (1/6) * M * sp.Pow(L,2)
I_yy = I_zz  = (1/12) * M * sp.Pow(L,2)
I_plate = sp.diag(I_xx, I_yy, I_zz)

T_linear_plate = 0.5 * M *  norm_sqrd(A_v_po)
T_angular_plate= 0.5 * sum(I_plate * elementwise_sqrd(A_omega_plate))

T_plate = T_linear_plate + T_angular_plate

### Kinetic energy of rods

In [10]:
I_xx = 0
I_yy = I_zz  = (1/12) * m * sp.Pow(L,2)
I_rod = sp.diag(I_xx, I_yy, I_zz)

T_linear_rod1 = 0.5 * m *  norm_sqrd(A_v_rod1)
T_linear_rod2 = 0.5 * m *  norm_sqrd(A_v_rod2)

T_angular_rod1 = 0.5 * sum(I_rod * elementwise_sqrd(A_omega_rod1))
T_angular_rod2 = 0.5 * sum(I_rod * elementwise_sqrd(A_omega_rod2))

T_rod1 = T_linear_rod1 + T_angular_rod1
T_rod2 = T_linear_rod2 + T_angular_rod2

### Total kinetic energy

In [11]:
T = T_plate + T_rod1 + T_rod2
T

## Potential energy

In [12]:
V_plate = M*g*L*sp.sin(beta)
V = V_plate
V

## Compute equation of motion

In [13]:
L = T - V

x = phi
xd = phi.diff(t)


EOM = sp.diff(sp.diff(L,xd), t) - sp.diff(L, x)

phi_eom = sp.simplify(EOM)
phi_eom

In [14]:
L = T - V

x =  theta
xd = theta.diff(t)


EOM = sp.diff(sp.diff(L,xd), t) - sp.diff(L, x)

phi_eom = sp.simplify(EOM)
phi_eom

In [15]:
x = (phi, theta)
xd = (phi.diff(t), theta.diff(t))
var = zip(x, xd)
eom_vec = []

for x,xd in var:
    eom = sp.diff(sp.diff(L,xd), t) - sp.diff(L, x)
    eom = sp.simplify(eom)
    
    display("Equation of motion for " + str(x) + "is: ")
    display(eom)

    eom_vec.append(eom)
    

'Equation of motion for phi(t)is: '

'Equation of motion for theta(t)is: '